# Uno multiplayer

#### This is the server and port variable, you need to change this to play on a specific server

In [211]:
PORT = 8080
SERVER = "localhost"

#### Import all library

In [212]:
import pygame
import socket
import time
import asyncio
import threading
import pickle
from datetime import datetime, timedelta

#### Button class

Taken at :
https://github.com/russs123/pygame_tutorials/blob/main/Button/button.py

By russs123 : 
https://github.com/russs123

In [213]:
class Button():
    def __init__(self, x, y, image, scale):
        width = image.get_width()
        height = image.get_height()
        self.image = pygame.transform.scale(image, (int(width * scale), int(height * scale)))
        self.rect = self.image.get_rect()
        self.rect.topleft = (x, y)
        self.clicked = False

    def draw(self, surface):
        action = False
        #get mouse position
        pos = pygame.mouse.get_pos()

        #check mouseover and clicked conditions
        if self.rect.collidepoint(pos):
            if pygame.mouse.get_pressed()[0] == 1 and self.clicked == False:
                self.clicked = True
                action = True

        if pygame.mouse.get_pressed()[0] == 0:
            self.clicked = False

        #draw button on screen
        surface.blit(self.image, (self.rect.x, self.rect.y))

        return action

#### Pygame initialisation

In [214]:
pygame.init()
screen = pygame.display.set_mode((1280,720)) #,pygame.FULLSCREEN
clock = pygame.time.Clock()
run = True
baseFont = pygame.font.Font(None,32)
pygame.display.set_caption('Uno multiplayer')

#### Pygame quit event

In [215]:
def quitEvent():
    run = False
    send(DISCONNECT_MESSAGE)
    pygame.quit()

#### Socket and var initialisation

In [216]:
HEADER = 64
FORMAT = 'utf-8'
DISCONNECT_MESSAGE = ['!DISCONNECT']
ADDR = (SERVER, PORT)
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
error = False
try:
    client.connect(ADDR)
except:
    error = True
connected = True
currentMessage = ["none","none","none"]
tmpMessage = ["none", "none", "none"]
currentRoom = ""
username = ""
players = []
myHand = []
heap = []
listNumber = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,18,14,15,16,17,19,20,21,22]
listColor = [0, 1, 2, 3, 4]   
listSpecialCard = [[1,4], [2,4], [3,4] , [4,4], [6,4] , [7,4] , [8,4] , [9,4]]
myTurn = False

#### Socket loop

In [217]:
def start_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

#### Socket get message

In [218]:
async def socket():
    global currentMessage
    global run
    global tmpMessage
    global myTurn
    while run:
        message = client.recv(512)
        currentMessage = pickle.loads(message)
        if currentMessage[len(currentMessage)-1] == "yourTurn":
            tmpMessage = currentMessage
            myTurn = True

#### Send function

In [219]:
def send(msg):
    message = pickle.dumps(msg)
    msg_lenght = len(message)
    send_lenght = str(msg_lenght).encode(FORMAT)
    send_lenght += b' ' * (HEADER - len(send_lenght))
    client.send(send_lenght)
    client.send(pickle.dumps(msg))

#### Load assets

In [220]:
newGameBtn = pygame.image.load('./assets/Buttons/Red/New Game 192x48.png').convert_alpha()
quitBtn = pygame.image.load('./assets/Buttons/Red/Quit 192x48.png').convert_alpha()
exitBtn = pygame.image.load('./assets/Buttons/Red/Exit 192x48.png').convert_alpha()
newBtn = pygame.image.load('./assets/Buttons/Red/New 192x48.png').convert_alpha()
joinBtn = pygame.image.load('./assets/Buttons/Red/Load Game 192x48.png').convert_alpha()
quitGameBtn = pygame.image.load('./assets/Buttons/Red/Quit Game 192x48.png').convert_alpha()
startGameBtn = pygame.image.load('./assets/Buttons/Red/Start Game 192x48.png').convert_alpha()
creditBtn = pygame.image.load('./assets/Buttons/Red/Credits 192x48.png').convert_alpha()
stack = pygame.image.load('./assets/Cards/Pile.png').convert_alpha()
spriteSheetCards = pygame.image.load('./assets/Cards/Cards.png').convert_alpha()
spriteSheetCardsGrayed = pygame.image.load('./assets/Cards/Cards.png').convert_alpha()
background = pygame.image.load('./assets/background.jpg').convert_alpha()
changeRed = pygame.image.load('./assets/Cards/ChangeRed.png').convert_alpha()
changeBlue = pygame.image.load('./assets/Cards/ChangeBlue.png').convert_alpha()
changeYellow = pygame.image.load('./assets/Cards/ChangeYellow.png').convert_alpha()
changeGreen = pygame.image.load('./assets/Cards/ChangeGreen.png').convert_alpha()
plus4Red = pygame.image.load('./assets/Cards/Plus4Red.png').convert_alpha()
plus4Blue = pygame.image.load('./assets/Cards/Plus4Blue.png').convert_alpha()
plus4Yellow = pygame.image.load('./assets/Cards/Plus4Yellow.png').convert_alpha()
plus4Green = pygame.image.load('./assets/Cards/Plus4Green.png').convert_alpha()
uno = pygame.image.load('./assets/Buttons/Red/Uno 192x48.png').convert_alpha()
counterUno = pygame.image.load('./assets/Buttons/Red/Counter Uno 192x48.png').convert_alpha()
arrowLeft = pygame.image.load('./assets/arrowLeft.png').convert_alpha()
arrowRight = pygame.image.load('./assets/arrowRight.png').convert_alpha()
hearc = pygame.image.load('./assets/hearcLogo.png').convert_alpha()
replayBtn = pygame.image.load('./assets/Buttons/Red/Restart Game 192x48.png').convert_alpha()

#### Load image of file 

In [221]:
def get_image(sheet, frameW, frameH, width, height, scale):
        image = pygame.Surface((width, height)).convert_alpha()
        image.blit(sheet, (0, 0), ((frameW * width), ((frameH * height)), width, height))
        image = pygame.transform.scale(image, (width * scale, height * scale))
                
        return image

#### Text Display

In [222]:
textCol = (255,255,255)
colorTextRect = (200,200,200)
def drawText(text, font, textCol, x, y):
    img = font.render(text, True, textCol)
    screen.blit(img, (x, y))

#### Join game menu

In [223]:
def joinMenu():
    
    global currentMessage

    global currentRoom

    global username

    errorMessage = ""
    
    localRun = True

    abilityToClick = False

    screen.fill((0, 0, 0))

    x, y = pygame.display.get_surface().get_size()
    
    #create rectangle as text box
    codeTextBox = pygame.Rect(x/2-((joinBtn.get_width()/2)*1.5),3*y/6-((newBtn.get_height()/2)*1.5)+ (newBtn.get_height()/2)-7, 105,joinBtn.get_height())
    usernameTextBox = pygame.Rect(x/2-((joinBtn.get_width()/2)*1.5),y/6-((newBtn.get_height()/2)*1.5)+ (newBtn.get_height()/2)-7, joinBtn.get_width()*1.5,joinBtn.get_height())
    #Variable for the text box (color of the rectangle and state of the box selected or not)
    activeCode = False
    activeUsername = False
    
    #String for the text box
    code = ""
    
    joinRoom = Button(x/2-((joinBtn.get_width()/2)*1.5), 4*y/6-((joinBtn.get_height()/2)*1.5), joinBtn, 1.5)
    newRoom = Button(x/2-((newBtn.get_width()/2)*1.5), 2*y/6-((newBtn.get_height()/2)*1.5), newBtn, 1.5)
    exitMenu = Button(x/2-((exitBtn.get_width()/2)*1.5), 5*y/6-((exitBtn.get_height()/2)*1.5), exitBtn, 1.5)
        
    
    while localRun:
        UI_REFRESH_RATE = clock.tick(60)/1000
        screen.blit(background, (0, 0))
        #draw the text box
        pygame.draw.rect(screen,colorTextRect,codeTextBox,2)
        pygame.draw.rect(screen,colorTextRect,usernameTextBox,2)
        #write text
        label = baseFont.render(errorMessage, 1, textCol)
        screen.blit(label, (x/2-((newBtn.get_width()/2)*1.5), y/9-((newBtn.get_height()/2)*1.5) + newBtn.get_height()/2))
        label = baseFont.render("Your username :", 1, textCol)
        screen.blit(label, (x/2-((newBtn.get_width()/2)*1.5) - 250, y/6-((newBtn.get_height()/2)*1.5) + newBtn.get_height()/2))
        label = baseFont.render("Room code :", 1, textCol)
        screen.blit(label, (x/2-((joinBtn.get_width()/2)*1.5) - 250, 3*y/6-((joinBtn.get_height()/2)*1.5) + joinBtn.get_height()/2))
        #set the location or where we can write
        drawText(username, baseFont, textCol, x/2-(newBtn.get_width()/2)*1.5 ,y/6-((newBtn.get_height()/2)*1.5)+ newBtn.get_height()/2)
        drawText(code, baseFont, textCol, x/2-(newBtn.get_width()/2)*1.5 ,3*y/6-((newBtn.get_height()/2)*1.5)+ newBtn.get_height()/2)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quitEvent()
            #check if a text box is selected
            if event.type == pygame.MOUSEBUTTONDOWN:
                if codeTextBox.collidepoint(event.pos):
                    activeCode = True
                else:
                    activeCode = False
                if usernameTextBox.collidepoint(event.pos): 
                    activeUsername = True
                else:
                    activeUsername = False
            #write in the code text box
            if activeCode == True:     
                if event.type == pygame.TEXTINPUT:
                    if len(code) < 5:
                        if event.text != "," and event.text != ".":
                            code += event.text.upper()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_BACKSPACE:
                        code = code[:-1]
            #write in the username text box            
            if activeUsername == True:     
                if event.type == pygame.TEXTINPUT:
                    if len(username) < 13:
                        if event.text != "," and event.text != ".":
                            username += event.text
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_BACKSPACE:
                        username = username[:-1]
        if newRoom.draw(screen) and abilityToClick:
            abilityToClick = False
            if len(username) > 0:
                send([f'{username}','createRoom',-1])
                while currentMessage[0] != "error" or currentMessage[0] != "joinNow":
                    if currentMessage[0] == "error":
                        currentRoom = currentMessage[1]
                        errorMessage = "Cannot create a room."
                        break
                    elif currentMessage[0] == "joinNow":
                        currentRoom = currentMessage[1]
                        errorMessage = ""
                        lobbyMenu(True)
                        break
                currentMessage = ["none","none","none"]
            else:
                errorMessage = "Please enter a username."
        if joinRoom.draw(screen) and abilityToClick:
            abilityToClick = False
            if len(username) > 0 and len(code) > 0:
                send([f'{username}','joinRoom',f'{code}'])
                while currentMessage[0] != "error" or currentMessage[0] != "joinNow":
                    if currentMessage[0] == "error":
                        currentRoom = currentMessage[1]
                        errorMessage = "Cannot join this room (may be full, not exist or username already in use)."
                        break
                    elif currentMessage[0] == "joinNow":
                        currentRoom = currentMessage[1]
                        errorMessage = ""
                        lobbyMenu(False)
                        break
                currentMessage = ["none","none","none"]
            else:
                errorMessage = "Please enter a username and a room code."
        if exitMenu.draw(screen) and abilityToClick:
            abilityToClick = False
            localRun = False
        abilityToClick = True
        pygame.display.flip()
        pygame.display.update()
        

#### Current lobby menu

In [224]:
def lobbyMenu(owner):
    localRun = True

    global players

    global currentRoom

    global currentMessage

    global username
    
    global myHand
    
    global heap
    
    abilityToClick = False
    
    errorMessage = ""

    x, y = pygame.display.get_surface().get_size()
    if owner:
        startGame = Button(x/2-((startGameBtn.get_width()/2)*1.5), y/3-((startGameBtn.get_height()/2)*1.5), startGameBtn, 1.5)
    quitGame = Button(x/2-((quitGameBtn.get_width()/2)*1.5), 2*y/3-((quitGameBtn.get_height()/2)*1.5), quitGameBtn, 1.5)
    send([f'{username}','getPlayers',f'{currentRoom}'])
    while localRun:
        screen.blit(background, (0, 0))
        label = baseFont.render(errorMessage, 1, textCol)
        screen.blit(label, (x/2-((newBtn.get_width()/2)*1.5), y/9-((newBtn.get_height()/2)*1.5) + newBtn.get_height()/2))
        if currentMessage[0] == "setPlayers":
            players = []
            counter = 0
            for msg in currentMessage:
                if counter > 0:
                    players.append(msg)
                counter = counter + 1
            currentMessage = ["none","none","none"]
        label = baseFont.render(f"Current room : {currentRoom}", 1, textCol)
        screen.blit(label, (50, 50))
        counter = 0
        if currentMessage[0] == "startNow":
            abilityToClick = False
            myHand = [[0,4],[0,4],[0,4],[2,0],[2,1],[2,2],[2,3]]  # à la place de ça -> currentMessage[2]
            heap = currentMessage[1]
            gameTable()
            currentMessage = ["none","none","none"]
        if currentMessage[0] == "notAValidNumberOfPerson":
            errorMessage = "You can't start the game. There are maybe some players missing."
            currentMessage = ["none","none","none"]
        for p in players:
            counter = counter + 1
            label = baseFont.render(f"Player {counter} : {p}", 1, textCol)
            screen.blit(label, (50, 50*(counter+1)))
        if owner:
            if startGame.draw(screen) and abilityToClick:
                abilityToClick = False
                send([f'{username}','startGame',f'{currentRoom}'])
        if quitGame.draw(screen) and abilityToClick:
            abilityToClick = False
            players = []
            send([f'{username}','quitRoom',f'{currentRoom}'])
            localRun = False            
        abilityToClick = True
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quitEvent()
        pygame.display.update()

#### End screen

In [225]:
def endScreen(message, winner):
    
    localRun = True
    
    abilityToClick = False
    
    global currentRoom
    
    global username
    
    global currentMessage
    
    global tmpMessage
    
    global players
    
    global myHand
    
    global heap
    
    x, y = pygame.display.get_surface().get_size()
    
    currentMessage = ["none","none","none"]
    
    errorMessage = ""
    
    newGame = Button(x/2-((newGameBtn.get_width()/2)*1.5), 4*y/6-((newGameBtn.get_height()/2)*1.5), newGameBtn, 1.5)
    quit = Button(x/2-((quitBtn.get_width()/2)*1.5), (y-quitBtn.get_height()-(quitBtn.get_height()/2))-20, quitBtn, 1.5)
    if message != "disconnected" and players[0] == username:
        replay = Button(x/2-((replayBtn.get_width()/2)*1.5), 3*y/6-((replayBtn.get_height()/2)*1.5),replayBtn,1.5)        
    
    while localRun:
        screen.blit(background, (0, 0))
        
        if currentMessage[0] == "partyEnd":
            currentMessage = ["none","none","none"]
            
        if currentMessage[0] == "updateHeap":
            currentMessage = ["none","none","none"]
            
        if tmpMessage[0] == "updateHeap":
            tmpMessage = ["none","none","none"]
        
        if currentMessage[0] == "setPlayers":
            endScreen("disconnected",username)
            currentMessage = ["none","none","none"]
            
        if currentMessage[0] == "startNow":
            abilityToClick = False
            myHand = currentMessage[2]  # à la place de ça -> currentMessage[2]
            heap = currentMessage[1]
            gameTable()
            currentMessage = ["none","none","none"]
            
        if currentMessage[0] == "error":
            currentMessage = ["none","none","none"]
        
        if message == "disconnected":
            label = baseFont.render(f"A player of your room disconnected. Room has been deleted.", 1, textCol)
            screen.blit(label, (x/2-label.get_rect().width/2, 2*y/6-((replayBtn.get_height()/2)*1.5)))
        else:
            label = baseFont.render(f"Winner of the room : {winner}", 1, textCol)
            replay = Button(x/2-((replayBtn.get_width()/2)*1.5), 3*y/6-((replayBtn.get_height()/2)*1.5),replayBtn,1.5)
            counter = 0
            screen.blit(label, (x/2-label.get_rect().width/2, 2*y/6-((replayBtn.get_height()/2)*1.5)))
            for p in players:
                counter = counter + 1
                label = baseFont.render(f"Player {counter} : {p}", 1, textCol)
                screen.blit(label, (50, 50*(counter+1)))
        if newGame.draw(screen) and abilityToClick:
            abilityToClick = False
            localRun = False
            send([f'{username}','leaveRoom',f'{currentRoom}'])
            mainMenu()
        if quit.draw(screen) and abilityToClick:
            abilityToClick = False
            localRun = False
            quitEvent()
        if message != "disconnected":
            if replay.draw(screen) and abilityToClick:
                abilityToClick = False
                send([f'{username}','startGame',f'{currentRoom}'])
        pygame.display.flip() 
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quitEvent()
        abilityToClick = True
        pygame.display.update()

#### Credits screen

In [226]:
def creditScreen():
    
    localRun = True
    
    abilityToClick = False
    
    global currentMessage
    
    x, y = pygame.display.get_surface().get_size()
    
    currentMessage = ["none","none","none"]
    
    exit = Button(x/2-((exitBtn.get_width()/2)*1.5), (y-exitBtn.get_height()-(exitBtn.get_height()/2))-20, exitBtn, 1.5)
    
    while localRun:
        screen.blit(background, (0, 0))
        label = baseFont.render("Cards designs : Esgow (reddit, u/Esgow)", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,10))
        label = baseFont.render("Buttons : russs123 (github)", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,60))
        label = baseFont.render("Buttons assets : Screaming Brain Studios (OpenGameArt)", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,110))
        label = baseFont.render("Oder assets : pixabay.com", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,160))
        label = baseFont.render("Server-side and server-client communication : Naddeo Eddy", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,210))
        label = baseFont.render("(nb: communication logic adapted for notebooks and arrays from Tech With Tim logic, youtube)", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,260))
        label = baseFont.render("Client side cards logic : Jaton David", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,310))
        label = baseFont.render("Client side UX : Jaton David and Naddeo Eddy", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,360))
        screen.blit(pygame.transform.scale(hearc,(849,159)),(x/2 - 849/2,410))
        if exit.draw(screen) and abilityToClick:
            abilityToClick = False
            localRun = False
        pygame.display.flip() 
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quitEvent()
        abilityToClick = True
        pygame.display.update()

#### Game table

In [227]:
def gameTable():
    
    localRun = True
    
    global currentMessage
    
    global tmpMessage
    
    global currentRoom
    
    global listNumber
    
    global listColor
    
    global myHand
    
    global heap
    
    global players
    
    global username
    
    global myTurn
    
    if players[0] != username:
        myTurn = False
    
    currentMessage = ["none","none","none"]
    
    abilityToClick = False
    
    changeColor = False
    
    plus4 = False
    
    x, y = pygame.display.get_surface().get_size()
    
    cardTmpInit = get_image(spriteSheetCards, listNumber[heap[0]] , listColor[heap[1]], 32, 48,3)
    
    cardX = cardTmpInit.get_width()
    cardY = cardTmpInit.get_height()
    
    stackX = x/2-((cardX/2)-(cardX))
    
    stackY = y/2-(cardY/2)
    
    heapX = x/2-((cardX/2)+(cardX))
    
    heapY = y/2-(cardY/2)
    
    stackButton = Button(stackX,stackY,stack,3)
    
    changeRedBtn    = Button(x/2-cardX/2-((cardX/2)+5), y/2-cardY/2-((cardY/2)+5), changeRed, 3)
    changeBlueBtn   = Button(x/2-cardX/2-((cardX/2)+5), y/2-cardY/2+((cardY/2)+5), changeBlue, 3)
    changeYellowBtn = Button(x/2-cardX/2+((cardX/2)+5), y/2-cardY/2-((cardY/2)+5), changeYellow, 3)
    changeGreenBtn  = Button(x/2-cardX/2+((cardX/2)+5), y/2-cardY/2+((cardY/2)+5), changeGreen, 3)
    
    plus4RedBtn    = Button(x/2-cardX/2-((cardX/2)+5), y/2-cardY/2-((cardY/2)+5), plus4Red, 3)
    plus4BlueBtn   = Button(x/2-cardX/2-((cardX/2)+5), y/2-cardY/2+((cardY/2)+5), plus4Blue, 3)
    plus4YellowBtn = Button(x/2-cardX/2+((cardX/2)+5), y/2-cardY/2-((cardY/2)+5), plus4Yellow, 3)
    plus4GreenBtn  = Button(x/2-cardX/2+((cardX/2)+5), y/2-cardY/2+((cardY/2)+5), plus4Green, 3)
    
    unoBtn = Button((5*x/6)+(x/12)-(uno.get_width()/2), y-(2*uno.get_height())-(uno.get_height()/2), uno,1)
    counterUnoBtn = Button((5*x/6)+(x/12)-(uno.get_width()/2), y-uno.get_height()-(uno.get_height()/2),counterUno,1)
    
    numberOfCards = []
        
    numberOfCardsTimeStamp = ["","","",""]
    
    inUno = [False, False, False, False]
    
    throughPlayers = True
    counter = 0
    currentIndex = 0
    playersOrder = []
    while throughPlayers:
        if counter == len(players) - 1:
            throughPlayers = False
        if currentIndex == len(players):
            currentIndex = 0
            playersOrder.append(players[currentIndex])
            counter = counter + 1
            currentIndex = currentIndex + 1
        else:
            playersOrder.append(players[currentIndex])
            counter = counter + 1
            currentIndex = currentIndex + 1
            
    currentPlayerPlaying = playersOrder[0]

    for player in players:
        numberOfCards.append(7)
        
    currentWay = False
    
    playerColor = (255,255,255)

    while localRun:
        
        notPlacable = False
        
        mousePosition = pygame.mouse.get_pos()
        
        screen.blit(background, (0, 0))
        
        rectangles = []
        
        if not myTurn :
            for i in range (0, len(myHand)):
                screen.blit(get_image(spriteSheetCards, listNumber[myHand[i][0]] , listColor[myHand[i][1]], 32, 48,3), (((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10))
                if (cardX * len(myHand)) < (x/6)*4:
                    rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, cardX, cardY)
                    rectangles.append(rect)
                    pygame.draw.rect(screen, (0,0,0), rect,  2)
                elif i != len(myHand)-1:
                    rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, (((x/6)*4))/len(myHand), cardY)
                    rectangles.append(rect)
                    pygame.draw.rect(screen, (0,0,0), rect,  2)
                else:
                    rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, cardX, cardY)
                    rectangles.append(rect)
                    pygame.draw.rect(screen, (0,0,0), rect,  2)
        else:
            for i in range (0, len(myHand)):
                if heap[1] != 4 and myHand[i][0] == heap[0] or myHand[i][1] == heap[1] or myHand[i][1] == 4 or \
                (heap == [1, 4] and myHand[i][1] == 0 and plus4 == False and changeColor == False) or \
                (heap == [2, 4] and myHand[i][1] == 1 and plus4 == False and changeColor == False) or \
                (heap == [3, 4] and myHand[i][1] == 2 and plus4 == False and changeColor == False) or \
                (heap == [4, 4] and myHand[i][1] == 3 and plus4 == False and changeColor == False) or \
                (heap == [6, 4] and myHand[i][1] == 0 and plus4 == False and changeColor == False) or \
                (heap == [7, 4] and myHand[i][1] == 1 and plus4 == False and changeColor == False) or \
                (heap == [8, 4] and myHand[i][1] == 2 and plus4 == False and changeColor == False) or \
                (heap == [9, 4] and myHand[i][1] == 3 and plus4 == False and changeColor == False) or \
                plus4 == True or changeColor == True:
                    screen.blit(get_image(spriteSheetCards, listNumber[myHand[i][0]] , listColor[myHand[i][1]], 32, 48,3), (((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10))
                    if (cardX * len(myHand)) < (x/6)*4:
                        rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, cardX, cardY)
                        rectangles.append(rect)
                        pygame.draw.rect(screen, (0,0,0), rect,  2)
                    elif i != len(myHand)-1:
                        rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, (((x/6)*4))/len(myHand), cardY)
                        rectangles.append(rect)
                        pygame.draw.rect(screen, (0,0,0), rect,  2)
                    else:
                        rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, cardX, cardY)
                        rectangles.append(rect)
                        pygame.draw.rect(screen, (0,0,0), rect,  2)
                else :
                    pygame.transform.grayscale(spriteSheetCards, spriteSheetCardsGrayed)
                    screen.blit(get_image(spriteSheetCardsGrayed, listNumber[myHand[i][0]] , listColor[myHand[i][1]], 32, 48,3), (((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10))
                    if (cardX * len(myHand)) < (x/6)*4:
                        rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, cardX, cardY)
                        rectangles.append(rect)
                        pygame.draw.rect(screen, (0,0,0), rect,  2)
                    elif i != len(myHand)-1:
                        rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, (((x/6)*4))/len(myHand), cardY)
                        rectangles.append(rect)
                        pygame.draw.rect(screen, (0,0,0), rect,  2)
                    else:
                        rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, cardX, cardY)
                        rectangles.append(rect)
                        pygame.draw.rect(screen, (0,0,0), rect,  2)
                
        if currentMessage[0] == "setPlayers":
            endScreen("disconnected",username)
            currentMessage = ["none","none","none"]
            
        if currentMessage[len(currentMessage)-1] == "takeCards":
            currentPlayerPlaying = currentMessage[4]
            for cards in currentMessage[5]:
                myHand.append(cards)
            send([f'{username}','updateNumberOfCards',f'{currentRoom}',len(myHand)])
            
        if currentMessage[0] == "startNow":
            currentMessage = ["none","none","none"]
        
        if myTurn:
            label = baseFont.render(f"It's your turn !", 1, textCol)
            labelX = label.get_width()
            screen.blit(label, (x/2 - labelX/2, y/2 - cardY - 50))
            
        if currentMessage[0] == "updateHeap":
            currentPlayerPlaying = currentMessage[4]
            heap=currentMessage[1]
            numberOfCards[playersOrder.index(currentMessage[2])] = currentMessage[3]
            if currentMessage[3] > 1:
                inUno[playersOrder.index(currentMessage[2])] = False
            numberOfCardsTimeStamp[playersOrder.index(currentMessage[2])] = datetime.now()
            if currentMessage[1] == [11,0] or currentMessage[1] == [11,1] or currentMessage[1] == [11,2] or currentMessage[1] == [11,3]:
                if currentWay == False:
                    currentWay = True
                else:
                    currentWay = False
            currentMessage = ["none","none","none"]
            
        if tmpMessage[0] == "updateHeap":
            currentPlayerPlaying = tmpMessage[4]
            heap=tmpMessage[1]
            numberOfCards[playersOrder.index(tmpMessage[2])] = tmpMessage[3]
            if tmpMessage[3] > 1:
                inUno[playersOrder.index(tmpMessage[2])] = False
            numberOfCardsTimeStamp[playersOrder.index(tmpMessage[2])] = datetime.now()
            tmpMessage = ["none","none","none"]
            
        if currentMessage[0] == "newCard":
            currentPlayerPlaying = currentMessage[4]
            myHand.append(currentMessage[3])
            numberOfCards[playersOrder.index(currentMessage[1])] = currentMessage[2]
            currentMessage = ["none","none","none"]
            
        if currentMessage[0] == "updateNumberOfCards":
            indexPlayerToUpdate = playersOrder.index(currentMessage[1])
            numberOfCards[playersOrder.index(currentMessage[1])] = currentMessage[2]
            if currentMessage[2] > 1:
                inUno[playersOrder.index(currentMessage[1])] = False
            numberOfCardsTimeStamp[playersOrder.index(currentMessage[1])] = datetime.now()
            currentMessage = ["none","none","none"]
            
        if currentMessage[0] == "partyEnd":
            endScreen("winner",currentMessage[1])
            
        if currentMessage[0] == "askedCard":
            currentPlayerPlaying = currentMessage[3]
            numberOfCards[playersOrder.index(currentMessage[1])] = currentMessage[2]
            if currentMessage[2] > 1:
                inUno[playersOrder.index(currentMessage[1])] = False
            currentMessage = ["none","none","none"]
            
        if currentMessage[0] == "takeCardsCounterUnoHit":
            for cards in currentMessage[1]:
                myHand.append(cards)
            send([f'{username}','updateNumberOfCards',f'{currentRoom}',len(myHand)])
            currentMessage = ["none","none","none"]
            
        if currentMessage[0] == "unoFromPlayer":
            inUno[playersOrder.index(currentMessage[1])] = True
            currentMessage = ["none","none","none"]
            
        currentIndex = players.index(username)
        
        throughPlayers = True
        
        counter = 0
        
        playersOrder = []
        
        while throughPlayers:
            if counter == len(players) - 1:
                throughPlayers = False
            if currentIndex == len(players):
                currentIndex = 0
                playersOrder.append(players[currentIndex])
                counter = counter + 1
                currentIndex = currentIndex + 1
            else:
                playersOrder.append(players[currentIndex])
                counter = counter + 1
                currentIndex = currentIndex + 1
        
        if len(myHand) > 1:
            inUno[playersOrder.index(username)] = False
            
        for i in range(0, len(players)):
            if currentPlayerPlaying == playersOrder[i]:
                playerColor = (0,255,0)
            else:
                playerColor = (255,255,255)
            if i == 1:
                for j in range(0,numberOfCards[i]):
                    screen.blit(pygame.transform.rotate(get_image(spriteSheetCards, listNumber[10], listColor[4], 32, 48,3),270), (-cardY/2+30,((j+1)*(y/6)*4)/(numberOfCards[i]+1)-(cardX/2)+(y/6)))
                if inUno[i] == True:
                    label = baseFont.render(f"{playersOrder[i]} - ICHI !", 1, playerColor)
                else:
                    label = baseFont.render(f"{playersOrder[i]}", 1, playerColor)
                screen.blit(pygame.transform.rotate(label,270), (cardY-30, y/2 - label.get_rect().width/2))
            elif i == 2:
                for j in range(0,numberOfCards[i]):
                    screen.blit(pygame.transform.rotate(get_image(spriteSheetCards, listNumber[10], listColor[4], 32, 48,3),180), (((j+1)*(x/6)*4)/(numberOfCards[i]+1)-(cardX/2)+(x/6), -cardY/2+30))
                if inUno[i] == True:
                    label = baseFont.render(f"{playersOrder[i]} - ICHI !", 1, playerColor)
                else:
                    label = baseFont.render(f"{playersOrder[i]}", 1, playerColor)
                screen.blit(label, (x/2 - label.get_rect().width/2, cardY-30))
            elif i == 3:
                for j in range(0,numberOfCards[i]):
                    screen.blit(pygame.transform.rotate(get_image(spriteSheetCards, listNumber[10], listColor[4], 32, 48,3),90), (x-cardY/2-30,((j+1)*(y/6)*4)/(numberOfCards[i]+1)-(cardX/2)+(y/6)))
                if inUno[i] == True:
                    label = baseFont.render(f"{playersOrder[i]} - ICHI !", 1, playerColor)
                else:
                    label = baseFont.render(f"{playersOrder[i]}", 1, playerColor)
                screen.blit(pygame.transform.rotate(label,90), (x-cardY+5, y/2 - label.get_rect().width/2))
        
        arrowX = arrowLeft.get_rect().width
        arrowY = arrowLeft.get_rect().height
        
        if currentWay == False:
            screen.blit(pygame.transform.scale(arrowLeft, (100, 100)), ((x/6)/2-50,y-cardY/2-60))
        else:
            screen.blit(pygame.transform.scale(arrowRight, (100, 100)), ((x/6)/2-50,y-cardY/2-60))
        
        screen.blit(get_image(spriteSheetCards, listNumber[heap[0]] , listColor[heap[1]], 32, 48,3), (heapX, heapY))
        pygame.draw.rect(screen, ((0,0,0)), pygame.Rect(heapX, heapY, cardX, cardY),  2)
        
        if stackButton.draw(screen) and abilityToClick and myTurn and plus4 == False and changeColor == False:
            abilityToClick = False
            myTurn = False
            send([f'{username}','demandingCard',f'{currentRoom}',len(myHand)+1])
        abilityToClick = True
        
        if len(myHand) == 1 and inUno[playersOrder.index(username)] == False and plus4 == False and changeColor == False:
            if unoBtn.draw(screen) : 
                send([f'{username}','uno',f'{currentRoom}'])
        clickable = True
        for player in playersOrder:
                if player != username and inUno[playersOrder.index(player)] == False and clickable and numberOfCards[playersOrder.index(player)] <= 1 and numberOfCardsTimeStamp[playersOrder.index(player)] + timedelta(seconds=5) < datetime.now():
                    if counterUnoBtn.draw(screen):
                        asValidCounterUno = False
                        for player in playersOrder:
                            if inUno[playersOrder.index(player)] == False and clickable and numberOfCards[playersOrder.index(player)] <= 1 and numberOfCardsTimeStamp[playersOrder.index(player)] + timedelta(seconds=5) < datetime.now():
                                asValidCounterUno = True
                                clickable = False
                                send([f'{username}','counterUno',f'{currentRoom}',f'{player}'])
                        if asValidCounterUno == False:
                            send([f'{username}','takeCardsCounterUno',f'{currentRoom}'])
        if changeColor == True:
            if changeRedBtn.draw(screen) :
                print("Red")
                myTurn = False
                changeColor = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[0],len(myHand)])
            elif changeBlueBtn.draw(screen) :  
                print("Blue")
                myTurn = False
                changeColor = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[1],len(myHand)])
            elif changeYellowBtn.draw(screen) :
                print("Yellow")
                myTurn = False
                changeColor = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[2],len(myHand)])
            elif changeGreenBtn.draw(screen) :
                print("Green")
                myTurn = False
                changeColor = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[3],len(myHand)])        
        if plus4 == True :
            if plus4RedBtn.draw(screen) :
                print("Red")
                myTurn = False
                plus4 = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[4],len(myHand)])
            elif plus4BlueBtn.draw(screen) :  
                print("Blue")
                myTurn = False
                plus4 = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[5],len(myHand)])
            elif plus4YellowBtn.draw(screen) :
                print("Yellow")
                myTurn = False
                plus4 = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[6],len(myHand)])
            elif plus4GreenBtn.draw(screen) :
                print("Green")
                myTurn = False
                plus4 = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[7],len(myHand)])
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.MOUSEBUTTONDOWN and changeColor == False and plus4 == False:
                if pygame.mouse.get_pressed()[0]:
                    for i in range (0, len(myHand)):
                        if rectangles[i].collidepoint(mousePosition) and myTurn:
                            if heap[1] != 4 and myHand[i][0] == heap[0] and myHand[i][1] != 4:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            elif myHand[i][1] == heap[1] and heap[1] != 4:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            elif myHand[i][1] == 4:
                                if myHand[i][0] == 0:
                                    changeColor = True
                                    myHand.pop(i)
                                elif myHand[i][0] == 5:
                                    plus4 = True
                                    myHand.pop(i)
                            elif plus4 == False and changeColor == False and heap == [1, 4] and myHand[i][1] == 0:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            elif plus4 == False and changeColor == False and heap == [2, 4] and myHand[i][1] == 1:                               
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            elif plus4 == False and changeColor == False and heap == [3, 4] and myHand[i][1] == 2:                     
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            elif plus4 == False and changeColor == False and heap == [4, 4] and myHand[i][1] == 3:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            elif plus4 == False and changeColor == False and heap == [6, 4] and myHand[i][1] == 0:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)  
                            elif plus4 == False and changeColor == False and heap == [7, 4] and myHand[i][1] == 1:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)  
                            elif plus4 == False and changeColor == False and heap == [8, 4] and myHand[i][1] == 2:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)  
                            elif plus4 == False and changeColor == False and heap == [9, 4] and myHand[i][1] == 3:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            else:
                                notPlacable = True
                            if len(myHand) == 0:
                                send([f'{username}','winning',f'{currentRoom}'])
                                endScreen("winner",username)
                                break
            if event.type == pygame.QUIT:
                quitEvent()
        pygame.display.update()

#### Server error screen

In [228]:
def errorScreen():
    
    localRun = True
    
    x, y = pygame.display.get_surface().get_size()

    while localRun:
        screen.blit(background, (0, 0))
        label = baseFont.render(f"The server \"{SERVER}\" with port \"{PORT}\" is not reachable.", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,y/2 - label.get_rect().height/2))
        pygame.display.flip() 
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        abilityToClick = True
        pygame.display.update()

#### Pygame main menu

In [229]:
def mainMenu():
    global run
    
    abilityToClick = False

    x, y = pygame.display.get_surface().get_size()
    newGame = Button(x/2-((newGameBtn.get_width()/2)*1.5), y/4-((newGameBtn.get_height()/2)*1.5), newGameBtn, 1.5)
    credits = Button(x/2-((creditBtn.get_width()/2)*1.5), y/2-((creditBtn.get_height()/2)*1.5), creditBtn, 1.5)
    quit = Button(x/2-((quitBtn.get_width()/2)*1.5), 3*y/4-((quitBtn.get_height()/2)*1.5), quitBtn, 1.5)

    while run:
        screen.blit(background, (0, 0))
        if newGame.draw(screen) and abilityToClick:
            abilityToClick = False
            joinMenu()
        if credits.draw(screen) and abilityToClick:
            abilityToClick = False
            creditScreen()
        if quit.draw(screen) and abilityToClick:
            abilityToClick = False
            quitEvent()
        abilityToClick = True
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quitEvent()
        pygame.display.update()
    

#### Start listening on socket

In [230]:
new_loop = asyncio.new_event_loop()
t = threading.Thread(target=start_loop, args=(new_loop,))
t.start()

asyncio.run_coroutine_threadsafe(socket(), new_loop)

<Future at 0x2bc328938e0 state=pending>

#### Pygame main loop

In [231]:
while run:

    screen.fill((0, 0, 0))
    
    x, y = pygame.display.get_surface().get_size()
    
    if error == True:
        errorScreen()
    else:
        mainMenu()

    pygame.display.flip()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False

    pygame.display.update()

quitEvent()

Blue


error: video system not initialized